In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Load Training Dataset
flights = pd.read_csv("flights.csv")
boston_flights = flights.loc[flights["ORIGIN"]=="BOS"]

print(len(boston_flights))

121275


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [40]:
# Feature Vector - 109 features (36 # and 73 categories)
# * MONTH * SCHED DEPART TIME * CARRIER * DEST AIRPORT *
features = np.zeros((len(boston_flights), 109))
carriers = ["B6", "AA", "DL", "UA", "WN", "NK", "EV", "VX", "AS", "OO"]
airports = boston_flights["DEST"].value_counts().index.tolist()


for i in range(0, len(boston_flights)):
    line = boston_flights.iloc[i]
    month = line.loc["MONTH"]
#     dow = line.loc["DAY_OF_WEEK"]
    departTime = int(line.loc["DEP_BLOCK"])
   
    carrier = line.loc["CARRIER"]
    carrierInd = carriers.index(carrier)
    
    destAirport = line.loc["DEST"]
    destInd = airports.index(destAirport)
    
    features[i,month-1]=1
#     features[i, dow+12-1] = 1
    features[i, departTime+12] = 1
    features[i, carrierInd+12+24-1] =1
    features[i, destInd+12+24+10-1] =1

In [46]:
# print("No delay: ", len(boston_flights[boston_flights["DEP_DELAY"]==0]))
# print("1-15 min delay: ", len(boston_flights[boston_flights["DEP_DELAY"].between(1,15, inclusive=True)]))
# print("15-30 min delay: ", len(boston_flights[boston_flights["DEP_DELAY"].between(15,29, inclusive=True)]))
# print("0.5-1 hour delay: ", len(boston_flights[boston_flights["DEP_DELAY"].between(30,59, inclusive=True)]))
# print("1-1.5 hour delay: ", len(boston_flights[boston_flights["DEP_DELAY"].between(60,89, inclusive=True)]))
# print("1.5-2 hour delay: ", len(boston_flights[boston_flights["DEP_DELAY"].between(90,119, inclusive=True)]))
# print("2-2.5 hour delay: ", len(boston_flights[boston_flights["DEP_DELAY"].between(120,149, inclusive=True)]))
print("0-30 min delay: ", len(boston_flights[boston_flights["DEP_DELAY"].between(0,29, inclusive=True)]))
print("0.5-1 hour delay: ", len(boston_flights[boston_flights["DEP_DELAY"].between(30,59, inclusive=True)]))
print("1-2 hour delay: ", len(boston_flights[boston_flights["DEP_DELAY"].between(60,119, inclusive=True)]))
print("Greater than 2 hour delay: ", len(boston_flights[boston_flights["DEP_DELAY"]>=120]))

0-30 min delay:  103793
0.5-1 hour delay:  7461
1-2 hour delay:  6147
Greater than 2 hour delay:  3874


In [42]:
# Label Vector 

# Classification Labels
labels = np.zeros((len(boston_flights), 4))
labels_single = np.zeros(len(boston_flights))
one, two, three, four = 0, 0, 0, 0

for i in range(0, len(boston_flights)):
    line = boston_flights.iloc[i]
    delay = line.loc["DEP_DELAY"]
    
    if (delay <= 30):
        delayBlock=0
        one+=1
        labels_single[i] = 0
        
    elif (30 < delay <= 60):
        delayBlock = 1
        two+=1
        labels_single[i] = 1
        
    elif (60 < delay <= 120):
        delayBlock = 2
        three+=1
        labels_single[i] = 2
        
    elif (delay > 120):
        delayBlock = 3
        four+=1
        labels_single[i] = 3

    labels[i,delayBlock] = 1

print(one,two,three, four)
print(labels_single)

104154 7285 6022 3814
[ 0.  0.  0. ...,  0.  0.  0.]


In [24]:
print(features.shape, labels.shape, labels_single.shape)

(121275, 109) (121275, 4) (121275,)


# 4-DelayBlock Classification

In [19]:
# Bagging
from sklearn.model_selection import KFold
from sklearn.ensemble import BaggingClassifier

kf = KFold(n_splits=4, shuffle=True)
for train_index, test_index in kf.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels_single[train_index], labels_single[test_index]
    
    print("Dataset split :", X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    clf = BaggingClassifier()
    print("Training.....")
    clf.fit(X_train,y_train)
    print("Testing.....")
    print("Score: ", clf.score(X_test, y_test)) 

Dataset split : (90956, 109) (30319, 109) (90956,) (30319,)
Training.....
Testing.....
Score:  0.855371219367
Dataset split : (90956, 109) (30319, 109) (90956,) (30319,)
Training.....
Testing.....
Score:  0.855272271513
Dataset split : (90956, 109) (30319, 109) (90956,) (30319,)
Training.....
Testing.....
Score:  0.856525611003
Dataset split : (90957, 109) (30318, 109) (90957,) (30318,)
Training.....
Testing.....
Score:  0.851012599776


In [20]:
#Logistic Regression

from sklearn.linear_model import LogisticRegression

kf = KFold(n_splits=4, shuffle=True)
for train_index, test_index in kf.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels_single[train_index], labels_single[test_index]
    
    print("Dataset split :", X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    clf = LogisticRegression()
    print("Training.....")
    clf.fit(X_train,y_train)
    print("Testing.....")
    print("Score: ", clf.score(X_test, y_test)) 

Dataset split : (90956, 109) (30319, 109) (90956,) (30319,)
Training.....
Testing.....
Score:  0.860120716382
Dataset split : (90956, 109) (30319, 109) (90956,) (30319,)
Training.....
Testing.....
Score:  0.858306672384
Dataset split : (90956, 109) (30319, 109) (90956,) (30319,)
Training.....
Testing.....
Score:  0.859494046637
Dataset split : (90957, 109) (30318, 109) (90957,) (30318,)
Training.....
Testing.....
Score:  0.857345471337


In [23]:
# Gradient Booster
from sklearn.ensemble import GradientBoostingClassifier

kf = KFold(n_splits=4, shuffle=True)
for train_index, test_index in kf.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels_single[train_index], labels_single[test_index]
    
    print("Dataset split :", X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    clf = GradientBoostingClassifier()
    print("Training.....")
    clf.fit(X_train,y_train)
    print("Testing.....")
    print("Score: ", clf.score(X_test, y_test)) 

Dataset split : (90956, 109) (30319, 109) (90956,) (30319,)
Training.....
Testing.....
Score:  0.858999307365
Dataset split : (90956, 109) (30319, 109) (90956,) (30319,)
Training.....
Testing.....
Score:  0.857745967875
Dataset split : (90956, 109) (30319, 109) (90956,) (30319,)
Training.....
Testing.....
Score:  0.856690524094
Dataset split : (90957, 109) (30318, 109) (90957,) (30318,)
Training.....
Testing.....
Score:  0.86183125536


In [22]:
# AdaBoost
from sklearn.ensemble import AdaBoostClassifier

kf = KFold(n_splits=4, shuffle=True)
for train_index, test_index in kf.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels_single[train_index], labels_single[test_index]
    
    print("Dataset split :", X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    clf = AdaBoostClassifier()
    print("Training.....")
    clf.fit(X_train,y_train)
    print("Testing.....")
    print("Score: ", clf.score(X_test, y_test)) 

Dataset split : (90956, 109) (30319, 109) (90956,) (30319,)
Training.....
Testing.....
Score:  0.860186681619
Dataset split : (90956, 109) (30319, 109) (90956,) (30319,)
Training.....
Testing.....
Score:  0.858405620238
Dataset split : (90956, 109) (30319, 109) (90956,) (30319,)
Training.....
Testing.....
Score:  0.858999307365
Dataset split : (90957, 109) (30318, 109) (90957,) (30318,)
Training.....
Testing.....
Score:  0.857708292104


In [21]:
#MLP
from sklearn.neural_network import MLPClassifier

kf = KFold(n_splits=4, shuffle=True)
for train_index, test_index in kf.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels_single[train_index], labels_single[test_index]
    
    print("Dataset split :", X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    clf = MLPClassifier()
    print("Training.....")
    clf.fit(X_train,y_train)
    print("Testing.....")
    print("Score: ", clf.score(X_test, y_test)) 

Dataset split : (90956, 109) (30319, 109) (90956,) (30319,)
Training.....
Testing.....
Score:  0.858207724529
Dataset split : (90956, 109) (30319, 109) (90956,) (30319,)
Training.....
Testing.....
Score:  0.857317193839
Dataset split : (90956, 109) (30319, 109) (90956,) (30319,)
Training.....
Testing.....
Score:  0.855931923876
Dataset split : (90957, 109) (30318, 109) (90957,) (30318,)
Training.....
Testing.....
Score:  0.858796754403


In [18]:
from sklearn.neighbors import KNeighborsClassifier

# Classification Test - KNN
kf = KFold(n_splits=4, shuffle=True)
for train_index, test_index in kf.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    print("Dataset split :", X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    knn = KNeighborsClassifier(n_neighbors=7)
    print("Training.....")
    knn.fit(X_train,y_train)
    print("Testing.....")
    print("Score: ",knn.score(X_test, y_test)) 

Dataset split : (90956, 109) (30319, 109) (90956, 4) (30319, 4)
Training.....
Testing.....
Score:  0.830238464329
Dataset split : (90956, 109) (30319, 109) (90956, 4) (30319, 4)
Training.....
Testing.....
Score:  0.830172499093
Dataset split : (90956, 109) (30319, 109) (90956, 4) (30319, 4)
Training.....
Testing.....
Score:  0.825785810878
Dataset split : (90957, 109) (30318, 109) (90957, 4) (30318, 4)
Training.....
Testing.....
Score:  0.829177386371


In [17]:
from sklearn.ensemble import RandomForestClassifier

# Classification Test - RF
kf = KFold(n_splits=4, shuffle=True)
for train_index, test_index in kf.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    print("Dataset split :", X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    clf = RandomForestClassifier()
    print("Training.....")
    clf.fit(X_train,y_train)
    print("Testing.....")
    print("Score: ", clf.score(X_test, y_test)) 

Dataset split : (90956, 109) (30319, 109) (90956, 4) (30319, 4)
Training.....
Testing.....
Score:  0.836999901052
Dataset split : (90956, 109) (30319, 109) (90956, 4) (30319, 4)
Training.....
Testing.....
Score:  0.843695372539
Dataset split : (90956, 109) (30319, 109) (90956, 4) (30319, 4)
Training.....
Testing.....
Score:  0.84224413734
Dataset split : (90957, 109) (30318, 109) (90957, 4) (30318, 4)
Training.....
Testing.....
Score:  0.83653275282


In [27]:
print(labels_single.shape)

(121275,)


In [28]:
#File Save
# boston_flights.to_csv("boston_flights.csv")
#np.savetxt("features.csv", features, delimiter=",")
# np.savetxt("labels_class.csv", labels_class, delimiter=",")
# np.savetxt("labels_reg.csv", labels_reg, delimiter=",")
np.savetxt("labels.csv", labels_single, delimiter=",")